<a href="https://colab.research.google.com/github/leomoraesguitar/separar_faixas2/blob/main/Separar_faixas_simplificada.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@markdown #Instalar bibliotecas (5min) - Rode duas vezes
!python3 -m pip install -U git+https://github.com/leomoraesguitar/separar_faixas2

import io
import os
from pathlib import Path
import select
from shutil import rmtree
import subprocess as sp
import sys
from typing import Dict, Tuple, Optional, IO

from google.colab import files

#@markdown #Funções






def find_files(in_path):
    out = []
    extensions = ["mp3", "wav", "ogg", "flac", "webm", "mp4"]
    for file in Path(in_path).iterdir():
        if file.suffix.lower().lstrip(".") in extensions:
            out.append(file)
    return out

def copy_process_streams(process: sp.Popen):
  def raw(stream: Optional[IO[bytes]]) -> IO[bytes]:
      assert stream is not None
      if isinstance(stream, io.BufferedIOBase):
          stream = stream.raw
      return stream

  p_stdout, p_stderr = raw(process.stdout), raw(process.stderr)
  stream_by_fd: Dict[int, Tuple[IO[bytes], io.StringIO, IO[str]]] = {
      p_stdout.fileno(): (p_stdout, sys.stdout),
      p_stderr.fileno(): (p_stderr, sys.stderr),
  }
  fds = list(stream_by_fd.keys())

  while fds:
    ready, _, _ = select.select(fds, [], [])
    for fd in ready:
        p_stream, std = stream_by_fd[fd]
        raw_buf = p_stream.read(2 ** 16)
        if not raw_buf:
            fds.remove(fd)
            continue
        buf = raw_buf.decode()
        std.write(buf)
        std.flush()

def separate(inp=None, outp=None):

    in_path = '/content'

    out_path = '/content/separados'



    if not os.path.exists(in_path):
      os.makedirs(in_path)
    if not os.path.exists(out_path):
      os.makedirs(out_path)
    inp = inp or in_path
    outp = outp or out_path
    model = "htdemucs_ft" # param ["htdemucs", "htdemucs_ft", "htdemucs_6s", "hdemucs_mmi", "mdx", "mdx_extra", "mdx_q", "SIG"]
    mp3_rate = 320
    files = [str(f) for f in find_files(inp)]
    if not files:
        print(f"No valid audio files in {in_path}")
        return

    cmd = ["python3", "-m", "demucs.separate", "-o", str(outp), "-n", model]

    cmd += ["--mp3", f"--mp3-bitrate={mp3_rate}"]


    print("Going to separate the files:")
    print('\n'.join(files))
    print("With command: ", " ".join(cmd))
    p = sp.Popen(cmd + files, stdout=sp.PIPE, stderr=sp.PIPE)
    copy_process_streams(p)
    p.wait()
    if p.returncode != 0:
        print("Command failed, something went wrong.")





In [ ]:
#@markdown #Rodar
#@markdown Click na pasta do lado esquerdo. Arraste a música desejada para lá. Depois execute esta célula

separate()

In [ ]:
#@markdown #Baixar faixas separadas
from google.colab import files
!zip -r separados.zip /content/separados
files.download('/content/separados.zip')

In [ ]:
#@markdown Mover Arquivos para o Drive
import shutil
import os

from google.colab import drive
drive.mount('/gdrive', force_remount=True)

if not os.path.exists('/gdrive/MyDrive/notebook_separar_faixas'):
  os.makedirs('/gdrive/MyDrive/notebook_separar_faixas')
%cd '/gdrive/MyDrive/notebook_separar_faixas'




def move_file(arquivo, destino):

  try:
    shutil.move(arquivo, destino)
    print(f'{arquivo} movido para a pasta "{destino}"')
  except:
    print(f'Deu erro ao mover o arquivo "{arquivo}"')


  """Moves a file from one location to another."""
  # source_path = os.path.join('/content/', arquivo)

  # destination_path = os.path.join('/content/drive/MyDrive/', self.destino)
  # destination_path = os.path.join('/content/drive/MyDrive/', self.destino)
  # if not os.path.exists(destination_path):
  #   print(f'A pasta "{destination_path}"  não existe. Criando pasta...')
  #   os.makedirs(destination_path)
def PastaDestino(destino):
  destination_path = os.path.join('/gdrive/MyDrive/demucs_separated/', destino)
  if not os.path.exists(destination_path):
    print(f'A pasta "{destination_path}"  não existe. Criando pasta...')
    os.makedirs(destination_path)
  return destination_path


# local = "/content/separados/htdemucs_ft" # @param {type:"string"}
local = os.path.join("/content/separados/",model)
# destino = "inflama" # @param {type:"string"}
destino = Path(local).name
#@markdown #A pasta será movida para a pasta "demucs_separated" do drive



current_dir = Path(local)

# List all directories in the current working directory
directories = [d for d in current_dir.iterdir() if d.is_dir()]

for j in directories:
  destination_path = PastaDestino(Path(j).name)

  for i in Path(j).iterdir():

    nome_saida = os.path.join(destination_path, Path(i).name)
    move_file(i, nome_saida)

